In [4]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [5]:
cd ..

/home/kgh6784/private/project/instance_segmentation


In [3]:
cd mmdetection

/home/kgh6784/private/project/instance_segmentation/mmdetection


In [3]:
import mmcv
import terminaltables
import pycocotools
from pycocotools.coco import COCO
from mmdet.datasets.builder import DATASETS
from mmdet.datasets import build_dataset
from mmdet.datasets.coco import CocoDataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.apis import init_detector, inference_detector
from mmdet.apis import show_result_pyplot
from mmcv.runner import load_checkpoint
from mmdet.apis import set_random_seed
import mmcv

In [4]:
import torch
import pylab
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
dataset_path = '/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset'

In [1]:
import numpy as np
from einops import rearrange
A = np.eye(10)
diag = np.einsum('ii->i', A)
trace =np.einsum('ii->', A)
print(diag)
print(trace)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
10.0


In [5]:
def _ntuple(n):
    def parse(x):
        if isinstance(x, container_abcs.Iterable):
            return x
        return tuple(repeat(x, n))

    return parse


to_1tuple = _ntuple(1)
to_2tuple = _ntuple(2)
to_3tuple = _ntuple(3)
to_4tuple = _ntuple(4)
print(to_1tuple, to_2tuple, to_3tuple, to_4tuple) 

<function _ntuple.<locals>.parse at 0x7f72bc1211f0> <function _ntuple.<locals>.parse at 0x7f72bc121280> <function _ntuple.<locals>.parse at 0x7f72bc121310> <function _ntuple.<locals>.parse at 0x7f72bc1213a0>


In [2]:
from itertools import repeat

In [4]:
tuple(repeat(3, 5))

(3, 3, 3, 3, 3)

In [ ]:
tuple(repeat(((2,),3)))

In [6]:
tuple(repeat(3,2))

NameError: name 'repeat' is not defined

In [6]:
# 사용할 모데 초기 py 설정
config = './configs/mask_rcnn/mask_rcnn_r50_fpn_1x_coco.py'

# config 불러오기
cfg = mmcv.Config.fromfile(config)

# 데이터셋 지정
cfg.dataset_type = 'CocoDataset'

# data_root는 사용자가 저장한 데이터가 있는 폴더 전까지
data_root = '/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset/'

# class 설정
cfg.classes = ('Normal',)

# train, val, test 데이터셋에 대한 type, data_root, ann_file, img_prefix 설정

cfg.data.samples_per_gpu=2
cfg.data.workers_per_gpu=1

cfg.data.train.data_root = data_root 
cfg.data.train.ann_file = 'modify.json' 
#cfg.data.train.ann_file = 'label(polygon)_train.json' 
cfg.data.train.img_prefix = 'train/'
cfg.data.train.classes = cfg.classes

cfg.data.val.data_root = data_root
#cfg.data.val.ann_file = 'label(polygon)_train.json'
cfg.data.train.ann_file = 'modify.json' 
cfg.data.val.img_prefix = 'train/'
cfg.data.val.classes = cfg.classes

cfg.data.test.data_root = data_root
cfg.data.test.ann_file = 'test.json'
cfg.data.test.img_prefix = 'test/'
cfg.data.test.classes = cfg.classes

# 클래스 수 지정
cfg.model.roi_head.bbox_head.num_classes = 1
cfg.model.roi_head.mask_head.num_classes = 1

# 사전 훈련 모델 지정
#cfg.load_from = './checkpoint/mask_rcnn_r50_fpn_1x_coco_20200205-d4b0c5d6.pth'
cfg.resume_from = './work_dirs/lg_cfg2/epoch_10.pth'
# 가중치 저장 위치
cfg.work_dir = './work_dirs/lg_cfg2/'

# learning rate
cfg.optimizer.lr = 0.02 / 8

# 로그 출력 시기 설정
cfg.log_config.interval = 1 
cfg.lr_config.policy = 'step'

# 평가 지표로 설정
cfg.evaluation.metric = ['bbox', 'segm']

# 평가 구간 설정

cfg.evaluation.interval = 10

# 체크포인트 구간 설정
cfg.checkpoint_config.interval = 10

# epoch 설정
cfg.runner = dict(type='EpochBasedRunner', max_epochs=20)
cfg.runner.max_epochs = 20
# 결과 재현을 위한 시드값 설정
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'



In [7]:
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='MaskRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_wei

In [ ]:
cfg.model

In [ ]:
cfg.evaluation.

In [ ]:
datasets = [build_dataset(cfg.data.train)]
model = build_detector(cfg.model)
model.CLASSES = datasets[0].CLASSES

# 훈련
train_detector(model, datasets, cfg, distributed=False, validate=True)

In [ ]:
img = mmcv.imread('/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset/test/0087.jpg')

model.cfg = cfg
result = inference_detector(model, img)
show_result_pyplot(model, img, result)

In [ ]:
cd mmdetection

In [ ]:
# 해당하는 폴더에 예측 json 저장 하는 코드입니다.

!python /home/kgh6784/private/project/instance_segmentation/mmdetection/test.py /home/kgh6784/private/project/instance_segmentation/mmdetection/configs/custom/lg_mask.py /home/kgh6784/private/project/instance_segmentation/mmdetection/work_dirs/lg_cfg2/latest.pth --format-only --eval-options "jsonfile_prefix=/home/kgh6784/private/project/instance_segmentation/mmdetection/work_dirs/lg_cfg2/"

In [ ]:
ls

In [ ]:
os.getcwd()

In [ ]:
os.listdir('/home/kgh6784/private/project/instance_segmentation/mmdetection/work_dirs/lg_cfg2/')

In [ ]:
import json

In [ ]:
with open("/home/kgh6784/private/project/instance_segmentation/mmdetection/work_dirs/lg_cfg2/.segm.json") as f:
    out_data2 = json.load(f)

In [ ]:
out_data2[0]

In [ ]:
result

In [ ]:
annFile = '/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset/label(polygon)_train.json'

In [ ]:
coco = COCO(annFile)

In [ ]:
cells = coco.loadCats(coco.getCatIds())
cells

In [ ]:
catids = coco.getCatIds(catNms=['Normal'])
print(catids)

In [ ]:
imgids = coco.getImgIds(catIds=catids)
print(imgids)

In [ ]:
img = coco.loadImgs(188)[0]
print(img)

In [ ]:
img['id']

In [ ]:
annids = coco.getAnnIds(imgIds=img['id'], catIds=catids, iscrowd=None)

In [ ]:
anns = coco.loadAnns(annids)

In [ ]:
anns

In [ ]:
ls

In [ ]:
file_path = '/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset/train/'+img['file_name']
image_array = cv2.imread(file_path)
image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
plt.imshow(image_array)
#plt.imshow(image_array)

In [ ]:
plt.figure(figsize=(12,14))
plt.imshow(image_array)
plt.axis('off')

coco.showAnns(anns)

In [ ]:
anns[2]['image_id']

In [ ]:
anns[2]['segmentation'] #|[0]

In [ ]:
len(anns)

In [ ]:
green_color = (0, 255, 0)

draw_img = image_array.copy()
polygon_all = []
for i in range(len(anns)):
    polygon_xy = np.array(anns[i]['segmentation'], np.int32)
    polygon_all.append(polygon_xy)
raw_img = cv2.polylines(draw_img, polygon_all, True, (0, 255, 0))

plt.figure(figsize=(12,14))
plt.imshow(draw_img)
plt.axis('off')

In [ ]:
polygon_all.shape

In [ ]:
check_img = image_array.copy()

In [ ]:
rot180.shape

In [ ]:
draw_img.shape

In [ ]:
flip_img = np.flip(check_img, 1)

In [ ]:
flip_img.shape

In [ ]:
plt.imshow(flip_img)

In [ ]:
#rot180 = np.rot90(np.rot90(check_img))
#plt.imshow(rot180)
raw_img = cv2.polylines(flip_img, polygon_all, True, (0, 255, 0))

plt.figure(figsize=(12,14))
plt.imshow(draw_img)
plt.axis('off')

In [ ]:
green_color = (0, 255, 0)

draw_img = image_array.copy()
raw_img = cv2.polylines(draw_img, polygon_all, True, (0, 255, 0))

plt.figure(figsize=(12,14))
plt.imshow(draw_img)
plt.axis('off')

In [ ]:
import json

In [ ]:
json_files = '/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset/label(polygon)_modify.json'

In [ ]:
with open(json_files) as json_file:
    json_data = json.load(json_file)

In [ ]:
for i in range(len(json_data['annotations'])):
    json_list = []
    for v in json_data['annotations'][i]['segmentation']:
        if v not in json_list:
            json_list.append(v)
    json_data['annotations'][i]['segmentation'] = json_list

In [ ]:
for i in range(len(json_data['annotations'])):
    json_list = [[]]
    segment_list = json_data['annotations'][i]['segmentation']
    for v in range(len(segment_list)):
        json_list[0].append(segment_list[v][0])
        json_list[0].append(segment_list[v][1])
    json_data['annotations'][i]['segmentation'] = json_list

In [ ]:
ttt = []
for i in range(len(json_data['annotations'])):
    img_id = json_data['annotations'][i]['image_id']
    if img_id not in ttt:
        ttt.append(img_id)

In [ ]:
xx = []
for i in range(len(json_data['annotations'])):
    ids = json_data['annotations'][i]['id']
    xx.append(ids)

In [ ]:
len(xx)

In [ ]:
xx

In [ ]:
json_data['annotations']#[1]['id']

In [ ]:
json_data

In [ ]:
a = [1,2,3,4]
a[:-1]

In [ ]:
json_data['annotations'][0]['segmentation'] =json_data['annotations'][0]['segmentation'][:-1]

In [ ]:
json_data['annotations']

In [ ]:
test = json_data['annotations'][0]['segmentation']

In [ ]:
test[0:3]

In [ ]:
x = [[]]
x[0]

In [ ]:
x = [[]]
for i in range(len(test)):
    x[0].append(test[i][0])
    x[0].append(test[i][1])

In [ ]:
x

In [ ]:
for i in range(len(test)):
    x_polygon = []
    y_polygon = []
    

In [ ]:
with open('/home/kgh6784/shared/hdd_ext/nvme1/jihun_instance/lg_molecular/dataset/modify.json','w') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

In [ ]:
x = 0
for i in range(len(json_data['annotations'])):
    json_data['annotations'][i]['id'] = x
    x += 1

In [ ]:
xx = []
for i in range(len(json_data['annotations'])):
    ids = json_data['annotations'][i]['id']
    xx.append(ids)

In [ ]:
xx